In [1]:
import numpy  as np 
import pandas as pd
from   glob   import glob
import itertools
import importlib
import time
from concurrent.futures import ProcessPoolExecutor

np.set_printoptions(edgeitems=30, linewidth=1000, formatter=dict(float=lambda x: "%.3g" % x))
np.set_printoptions(precision=8)

import talib
from data_provider import read_data_path, read_prices, read_data

In [2]:
#vectorized simulation
from TA_robots import rsi
robot = rsi()
params = robot.ps_ranges

prices = read_prices('BNB-BTC', '15m')
signals = robot.signals(prices, 16, 30, 74)

def fastsim(prices, signals, init_data):
    commission      = init_data['exchange_commission']
    start_capital   = init_data['start_capital']
    buy_correction  = init_data['buy_correction']
    sell_correction = init_data['sell_correction']

    s1 = signals[signals!=0]

    if len(s1) == 0:
        profit = 0
        trades = 0
        profit_to_bnh = 0
    else:
        inds = np.where(signals!=0)
        s2 = s1 + 1
        s3 = np.roll(s2,1)
        s3[0] = 0
        s4 = np.logical_xor(s2,s3)
        inds1  = inds[0][s4]
        trades = int(len(inds1)/2)
        inds2 = inds1[:trades*2]
        correction = np.tile([buy_correction,sell_correction], trades)
        start   = len(prices) - len(signals) 
        prices1 = prices[start:]
        prices2 = prices1[inds2]
        prices3 = prices2 * correction
        powers = np.tile([-1, 1], trades)
        cum_trades  = np.cumprod(np.power(prices3, powers)*(1-commission)) * start_capital

        profit = cum_trades[-1] - start_capital  if len(cum_trades) != 0 else 0
        bnh = prices[-1] / prices[0] * start_capital - start_capital
        profit_to_bnh = round(profit/bnh, 2)
   # return correction
    return [profit,trades,profit_to_bnh]


In [3]:
module  = importlib.import_module('TA_robots')
rnames = module.test
#rnames = ['rsi']
robots = []
for robot_name in rnames:
    cl = getattr(module, robot_name)
    robot = cl()
    robots.append(robot)

init_data = {'exchange_commission': 0.00075,
             'start_capital'     : 100,
             'buy_correction'    : 1.004,
             'sell_correction'   : 0.996,
             'robot_name'        : '',
             'market'            : '',
             'kline'             : ''  
            }

data_path = read_data_path()
pair = 'BTC'
#markets = glob(data_path + '/' + pair +'/*-' + pair)
#markets = [x.split('/')[-1] for x in markets]
fast_trend = ['ADA-BTC', 'EOS-BTC', 'TRX-BTC', 'ONT-BTC', 'NEO-BTC','ZIL-BTC', 'XRP-BTC'] #april 2018
markets = fast_trend
#markets = ['XLM-BTC']
timeframes = ['3m', '5m',  '15m', '24m', '30m', '1h']
timeframes = ['3m', '5m',  '15m', '24m', '30m', '1h', '2h', '4h','6h']
#timeframes = ['5m']

def fo(params):
    signals = robot.signals(prices, *params)

    ts = fastsim(ticks, signals, init_data)
    res = [init_data['market'], init_data['kline']] + ts + [init_data['robot_name']] + list(params)
 
    return res #Don't remove

def trade_statistics(prices, trade_data, init_data):
    #trade statistics        
    profit = trade_data.iloc[-1]['cum_profit']       
    s = trade_data['signal']
    trades = s[s==-1].shape[0] 
    bnh = prices[-1] / prices[0] * init_data['start_capital']
    profit_to_bnh = round(profit/bnh, 2)
    
    ts = [init_data['market'], init_data['kline'], profit, trades, profit_to_bnh]
    return ts

def cp_gen(params):
    pr = []
    for p in params:
        pr.append(np.arange(*p))
        
    pl = [x for x in itertools.product(*pr)]        
    return pl

In [4]:
train_split = 0.5
from_date = '2018-04-01'
to_date = '2018-05-05'

all_res = []
start_time_all = time.time() 
for robot in robots:
    robot_name = robot.robot_name
    print(robot_name, ": ", robot.comment)
    params = robot.ps_ranges 
    params_list = cp_gen(params)
    for market in markets:
        print('\t ', market)
        for kline in timeframes:
            print('\t\t', kline)
            data = read_data(market, kline, from_date, to_date)
            train = int(train_split * data.shape[0])
            prices = data[:train]
            ticks = prices['C'].values
            
            init_data['robot_name'] = robot.robot_name
            init_data['market'] = market
            init_data['kline'] = kline
            start = time.time()
            pool = ProcessPoolExecutor(max_workers = 8)
            with pool as p:            
                all_res = list(p.map(fo, params_list))
            #print(market, kline,  "Time taken = {0:.5f}".format(time.time() - start))   
            
            df_train_res = pd.DataFrame(all_res)
            df_train_res = df_train_res.sort_values(2, ascending = False)
            prices = data[train:]
            ticks = prices['C'].values
            res = []
            for ind in range(10):
                params = tuple(df_train_res.iloc[ind,6:])  
                signals = robot.signals(prices, *params)
                ts = fastsim(ticks, signals, init_data)
                res.append([init_data['market'], init_data['kline']] + ts + [init_data['robot_name']] + list(params))
                
            df_res = pd.DataFrame(res)
            df_res.to_csv('Logs/res_' + robot_name + '_' + market +'_' + kline +'.csv', index = False)
print("All done = {0:.2f} min".format((time.time() - start_time_all)/60))

rsi__ma :  
	  ADA-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  EOS-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  TRX-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  ONT-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  NEO-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  ZIL-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  XRP-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
ma-rsi-01 :  
	  ADA-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  EOS-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  TRX-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  ONT-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  NEO-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  ZIL-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
	  XRP-BTC
		 3m
		 5m
		 15m
		 24m
		 30m
		 1h
		 2h
		 4h
		 6h
All done = 6.72 min


In [12]:
files = glob('Logs/res_*-BTC*.csv')
all_res = []
for fn in files:
    dfx = pd.read_csv(fn)
    all_res.append(dfx)
df = pd.concat(all_res) 
df.columns = ['market', 'tf', 'profit', 'trades', 'profit_to_bnh', 'robot','p1','p2','p3','p4']
df['robot_set'] = df['robot'] + df['p1'].astype('str') + df['p2'].astype('str') + df['p3'].astype('str')
df = df[df['trades']>10]
#df = df[df['robot'] == 'ma-rsi-01']
df.sort_values('profit', ascending= False)
xx = df.groupby(['tf','robot']).sum()
xx.sort_values('profit', ascending= False)


profit  trades  profit_to_bnh     p1  \
tf  robot                                                                   
3m  ma-rsi                   1974.612690538     688          22.33   1906   
15m ma-rsi                   1851.241785466     420          35.55   1722   
3m  rsi                       980.466399163     840           6.64    790   
5m  ma-rsi                    960.755991754     862          21.37   1280   
3m  ma-rsi-01                 856.476958305     390          14.67    800   
5m  rsi                       848.644716240     697          12.68    524   
    ma-rsi-01                 799.432913426     390          13.88    294   
24m ma-rsi                    642.883947754     163          10.82    648   
5m  kestner-ma                617.593891739    1469         -34.95   2588   
15m macd                      574.827921688    1330         -23.80    887   
5m  rsi__ma                   574.781047598     457           8.22    308   
24m ma                        537.689735806    1081         -57.36  10218   
30m ma                        494.841662268     955         -64.78   7360   
1h  ma-rsi                    487.869671050      86           7.24    504   
30m ma-rsi                    375.574335883      55           5.54    354   
2h  ma-rsi                    361.335132170      58           5.37    160   
24m rsi                       305.123904208      84           4.60     40   
    macd                      266.273302974     504         -33.26    393   
15m rsi                       213.578949288     199           2.42     94   
5m  ma-2-crossover-ichimoku   167.159906350    1423         -32.23   3232   
24m ma-rsi-01                 139.654747107      24           2.10      8   
1h  ma-3-crossover            135.721061751     362         -14.07    280   
30m ma-3-crossover-ichimoku   117.683098333     435         -26.76    533   
    rsi                       101.042996205     189          -2.03     46   
    ma-3-crossover             81.870194735     547         -33.84    753   
15m ma-rsi-01                  75.633422258      11           1.13      6   
3m  rsi__ma                    68.228717111     479         -12.89    424   
1h  rsi                        47.986513683      14           0.71      4   
    ma-2-crossover             32.934348494      46           1.96     48   
    ma-3-crossover-ichimoku    30.547035171     119          -9.84    123   
...                                     ...     ...            ...    ...   
24m ma-2-crossover-ichimoku   -11.529609518      14          -1.08     26   
4h  ma                        -13.008857055      60          -4.13     52   
30m ma-2-crossover-ichimoku   -13.261198302      42          -1.84     78   
24m kestner-ma                -20.510376802      24          -4.96    156   
5m  ma-2-crossover            -29.465274324    1341         -25.35   3346   
15m rsi__ma                   -31.046715545     283          -2.73     96   
5m  ma-3-crossover-ichimoku   -46.313703533     574          -0.59    419   
15m ma-2-crossover-ichimoku   -51.031811873     239         -13.09    930   
30m rsi__ma                   -53.687594864     172          -3.43     68   
24m rsi__ma                   -62.997013795     187          -7.62     60   
5m  ma-3-crossover            -73.464457388     384          -1.14    226   
24m ma-3-crossover-ichimoku  -114.376209057     728         -32.16    763   
15m ma-3-crossover           -115.810770575    1187         -36.56   1333   
2h  rsi-ma                   -158.677027856     250          -9.37    230   
3m  ma-2-crossover           -201.767349195     869         -17.09    994   
15m ma-3-crossover-ichimoku  -235.751013969     905         -38.52   1218   
24m ma-3-crossover           -243.179969347    1124         -46.78   1089   
15m ma                       -392.157221715    1902         -98.01   9784   
1h  rsi-ma                   -395.756011662    1189         -56.35    714   
3m  ma-2-crossover-ichimoku  -545

In [13]:
df[(df['tf'] == '3m') & (df['robot'] == 'ma-rsi')]

,market,tf,profit,trades,profit_to_bnh,robot,p1,p2,p3,p4,robot_set
0,ONT-BTC,3m,94.117620805,11,1.03,ma-rsi,24,22.0,78.0,NaN,ma-rsi2422.078.0
1,ONT-BTC,3m,99.125893078,11,1.09,ma-rsi,66,22.0,78.0,NaN,ma-rsi6622.078.0
2,ONT-BTC,3m,99.125893078,11,1.09,ma-rsi,70,22.0,78.0,NaN,ma-rsi7022.078.0
3,ONT-BTC,3m,99.396654782,11,1.09,ma-rsi,62,22.0,78.0,NaN,ma-rsi6222.078.0
4,ONT-BTC,3m,99.125893078,11,1.09,ma-rsi,76,22.0,78.0,NaN,ma-rsi7622.078.0
5,ONT-BTC,3m,99.125893078,11,1.09,ma-rsi,74,22.0,78.0,NaN,ma-rsi7422.078.0
6,ONT-BTC,3m,99.125893078,11,1.09,ma-rsi,72,22.0,78.0,NaN,ma-rsi7222.078.0
7,ONT-BTC,3m,99.396654782,11,1.09,ma-rsi,60,22.0,78.0,NaN,ma-rsi6022.078.0
8,ONT-BTC,3m,99.125893078,11,1.09,ma-rsi,64,22.0,78.0,NaN,ma-rsi6422.078.0
9,ONT-BTC,3m,99.125893078,11,1.09,ma-rsi,68,22.0,78.0,NaN,ma-rsi6822.078.0


In [19]:
files = glob('Logs/res_*-BTC*.csv')
all_res = []
for fn in files:
    dfx = pd.read_csv(fn)
    all_res.append(dfx)
df = pd.concat(all_res) 
df.columns = ['market', 'tf', 'profit', 'trades', 'profit_to_bnh', 'robot','p1','p2','p3','p4']
df['robot_set'] = df['robot'] + df['p1'].astype('str') + df['p2'].astype('str') + df['p3'].astype('str')

xx = df.groupby(['robot_set', 'tf']).sum()
xx.sort_values('profit', ascending= False)

,,profit,trades,profit_to_bnh,p1,p2,p3,p4
robot_set,tf,,,,,,,
ma-rsi822.078.0,30m,166.674203130,9,2.56,16,44.0,156.0,0.0
ma-rsi7022.078.0,3m,166.550016912,38,4.95,210,66.0,234.0,0.0
kestner-ma2414.010.0,6h,166.286604028,4,2.53,96,56.0,40.0,20.0
ma-rsi422.078.0,5m,157.497371319,19,2.35,8,44.0,156.0,0.0
ma-rsi3622.078.0,1h,156.145775606,5,2.40,72,44.0,156.0,0.0
ma-rsi3422.078.0,1h,151.818910984,5,2.33,68,44.0,156.0,0.0
ma-rsi814.078.0,6h,151.705417807,3,1.61,8,14.0,78.0,0.0
rsi1838.078.0,3m,151.674091230,43,3.27,54,114.0,234.0,0.0
rsi1438.076.0,24m,151.256186388,13,1.11,42,114.0,228.0,0.0


In [20]:
df[(df['tf'] == '3m') & (df['robot_set'] == 'ma-rsi7022.078.0')]

,market,tf,profit,trades,profit_to_bnh,robot,p1,p2,p3,p4,robot_set
2,ONT-BTC,3m,99.125893078,11,1.09,ma-rsi,70,22.0,78.0,NaN,ma-rsi7022.078.0
8,NEO-BTC,3m,13.173114289,6,3.02,ma-rsi,70,22.0,78.0,NaN,ma-rsi7022.078.0
5,EOS-BTC,3m,54.251009544,21,0.84,ma-rsi,70,22.0,78.0,NaN,ma-rsi7022.078.0


In [84]:
market = 'ONT-BTC'
kline  = '3m'
data = read_prices(market, kline, from_date, to_date)

train = int(train_split * data.shape[0])
prices = data[:train]

robot = robots[5]
signals = robot.signals(prices, 24,4,55)


In [85]:

simulation(prices, signals, init_data)




,price,signal,MP,capital,num_shares,profit,cum_profit
0,0.00029926,1,1,0.000000000,332659.870442710,-100.000000000,0.000000000
1,0.00029951,0,1,0.000000000,332659.870442710,-100.000000000,0.000000000
2,0.00030136,-1,0,99.799452354,0.000000000,-0.200547646,-0.200547646
3,0.00030184,0,0,99.799452354,0.000000000,-0.200547646,-0.200547646
4,0.00030004,1,1,0.000000000,331129.662883230,-99.799452354,-0.200547646
5,0.00029900,0,1,0.000000000,331129.662883230,-99.799452354,-0.200547646
6,0.00029960,0,1,0.000000000,331129.662883230,-99.799452354,-0.200547646
7,0.00029862,0,1,0.000000000,331129.662883230,-99.799452354,-0.200547646
8,0.00029961,0,1,0.000000000,331129.662883230,-99.799452354,-0.200547646
9,0.00029923,0,1,0.000000000,331129.662883230,-99.799452354,-0.200547646


In [75]:
rsi_win = 14
rsi_sell = 78
rsi = talib.RSI(prices * 100, rsi_win)
rsi[:rsi_win] = 0
sum(rsi>rsi_sell)

145

In [62]:
def simulation(ticks, signals, init_data):
    market          = init_data['market']
    commission      = init_data['exchange_commission']
    start_capital   = init_data['start_capital']
    buy_correction  = init_data['buy_correction']
    sell_correction = init_data['sell_correction']
     
    cur_capital  = start_capital 
    num_shares   = 0
    trade_profit = 0
    cum_profit   = 0
    
    market_position = 0
    start = len(ticks) - len(signals) 
    ticks = ticks[start:]
    trade_statistics = []
    #trade_statistics = np.zeros((len(signals),7))
    for i in range(len(signals)) :
        
        if market_position == 0 and signals[i] == 1:
            price = ticks[i] * buy_correction
            num_shares   = cur_capital * (1-commission)/price
            trade_profit = -cur_capital
            cur_capital  = 0
            market_position = 1
            
        elif market_position == 1 and signals[i] == -1:
            price = ticks[i] * sell_correction
            cur_capital  = (num_shares * price)*(1-commission)
            trade_profit = trade_profit + cur_capital
            cum_profit  += trade_profit
            num_shares   = 0
            market_position = 0
        else:
            signals[i] = 0
            
        trade_statistics.append([ticks[i], signals[i], market_position, cur_capital, num_shares, trade_profit, cum_profit])            
        #trade_statistics[i] = [ticks[i], signals[i], market_position, cur_capital, num_shares, trade_profit, cum_profit]

    df = pd.DataFrame(trade_statistics)
    df.columns = ['price', 'signal', 'MP', 'capital', 'num_shares', 'profit', 'cum_profit']
    df = df.astype({'signal':np.int32, 'MP':np.int32})
    return df
    #return 0

In [ ]:
def simulation_short(ticks, signals, init_data):
    market          = init_data['market']
    commission      = init_data['exchange_commission']
    start_capital   = init_data['start_capital']
    buy_correction  = init_data['buy_correction']
    sell_correction = init_data['sell_correction']
     
    cur_capital  = start_capital 
    num_shares   = 0
    trade_profit = 0
    cum_profit   = 0
    
    market_position = 0
    start = len(ticks) - len(signals) 
    ticks = ticks[start:]
    trade_statistics = []
    #trade_statistics = np.zeros((len(signals),7))
    for i in range(len(signals)) :
        
        #long entry
        if market_position == 0 and signals[i] == 1:
            price = ticks[i] * buy_correction
            num_shares   = cur_capital * (1-commission)/price
            trade_profit = -cur_capital
            cur_capital  = 0
            market_position = 1
        #long exit
        elif market_position == 1 and signals[i] == -1:
            price = ticks[i] * sell_correction
            cur_capital  = (num_shares * price)*(1-commission)
            trade_profit = trade_profit + cur_capital
            cum_profit  += trade_profit
            num_shares   = 0
            market_position = 0
        #short entry
        elif market_position == 0 and signals[i] == -1:
            price = ticks[i] * sell_correction
            num_shares   = cur_capital * (1-commission)/price
            trade_profit = num_shares * price
            market_position = 2
        #short exit
        elif market_position == 2 and signals[i] == 1:    
            price = ticks[i] * buy_correction
            cur_capital  = (num_shares * price)*(1-commission)
            num_shares = 0
            trade_profit = trade_profit - cur_capital
            market_position = 0
        else:
            signals[i] = 0
            
        trade_statistics.append([ticks[i], signals[i], market_position, cur_capital, num_shares, trade_profit, cum_profit])            
        #trade_statistics[i] = [ticks[i], signals[i], market_position, cur_capital, num_shares, trade_profit, cum_profit]

    df = pd.DataFrame(trade_statistics)
    df.columns = ['price', 'signal', 'MP', 'capital', 'num_shares', 'profit', 'cum_profit']
    df = df.astype({'signal':np.int32, 'MP':np.int32})
    return df
    #return 0